In [58]:
import tensorflow as tf
from tensorflow.keras.layers import Layer,DepthwiseConv2D, Conv2D, MaxPooling2D, BatchNormalization,LeakyReLU, Input, Flatten, Dense

In [100]:
class bn_ac(Layer):
    def __init__(self, trainable=True):
        super(bn_ac, self).__init__(trainable=trainable)
        self.bn = BatchNormalization()
        self.ac = LeakyReLU()
    
    def call(self,input):
        x = self.bn(input)
        x = self.ac(x)
        return x

In [101]:
class MobBlock(Layer):
    def __init__(self,input_channels,output_channels, trainable=True):
        super(MobBlock,self).__init__(trainable=trainable)
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.conv1 = Conv2D(self.input_channels, kernel_size=(1, 1), padding='same', use_bias=False)
        self.bn_ac1 = bn_ac()
        self.depthwise1 = DepthwiseConv2D(kernel_size=(1,3), padding='same', use_bias=False)
        self.bn_ac2 = bn_ac()
        self.maxpool = MaxPooling2D(pool_size=(2,1), strides=(2,1))
        self.depthwise2 = DepthwiseConv2D(kernel_size=(3,1), padding='same', use_bias=False)
        self.bn_ac3 = bn_ac()
        self.conv2 = Conv2D(self.output_channels, kernel_size=(2,1), strides=(1,2), padding='same', use_bias=False)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn_ac1(x)
        x = self.depthwise1(x)
        x = self.bn_ac2(x)
        x = self.maxpool(x)
        x = self.depthwise2(x)
        x = self.bn_ac3(x)
        x = self.conv2(x)
        return x

In [102]:
class EfficientNet(tf.keras.Model):
    def __init__(self,num_classes):
        super(EfficientNet, self).__init__()
        self.mob1 = MobBlock(32, 64)
        self.mob2 = MobBlock(64, 128)
        self.mob3 = MobBlock(128, 256)
        self.flatten = Flatten()
        self.dense = Dense(num_classes, activation='softmax')
    
    def call(self, inputs):
        x = self.mob1(inputs)
        x = self.mob2(x)
        x = self.mob3(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

In [103]:
model = EfficientNet(100)

In [104]:
model.build((None,224,224,3))

In [105]:
model.summary()

Model: "efficient_net_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mob_block_21 (MobBlock)      multiple                  4768      
_________________________________________________________________
mob_block_22 (MobBlock)      multiple                  21632     
_________________________________________________________________
mob_block_23 (MobBlock)      multiple                  84224     
_________________________________________________________________
flatten_8 (Flatten)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  20070500  
Total params: 20,181,124
Trainable params: 20,179,780
Non-trainable params: 1,344
_________________________________________________________________
